In [1]:
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3
import pandas as pd
import numpy as np
import io
from sagemaker.session import Session
from sagemaker import get_execution_role
import matplotlib.pyplot as plt
sagemaker_session = Session()

feature_group = FeatureGroup(name = 'BtcPriceData')

query = feature_group.athena_query()

table = query.table_name
table
default_s3_bucket_name = 'binance-btc-prices'
n_steps = 24

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [34]:
q= f'''
SELECT *
FROM
    (SELECT *,
    row_number()
    OVER
        (PARTITION BY "open_time"
        ORDER BY "close_time" DESC, Api_Invocation_Time DESC, write_time DESC)
    AS row_number
    FROM "sagemaker_featurestore"."btcpricedata_1704246894")
WHERE row_number = 1 ORDER BY close_time desc limit {n_steps};
'''

In [77]:
# run Athena query. The output is loaded to a Pandas dataframe.
dataset = pd.DataFrame()
query.run(query_string=q, output_location='s3://'+default_s3_bucket_name+'/query_results/')
query.wait()
dataset = query.as_dataframe()

2024-01-11 20:58:47,931 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:58:52,985 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:58:58,052 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:59:03,095 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:59:08,156 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:59:13,206 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:59:18,249 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:59:23,295 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:59:28,351 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 is being executed.
2024-01-11 20:59:33,395 INFO: Query 5a695440-d116-471b-84fd-36cfb22d3bc2 successfully executed.


In [29]:
import hopsworks
import joblib
project = hopsworks.login()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) DD2223_lab1
	 (2) project0



Enter project to access:  1



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/206996


In [5]:
mr = project.get_model_registry()
model = mr.get_model("mlpModel", version=2)
model_dir = model.download()
#model = joblib.load(model_dir + "/lstm_model.h5")

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
import os
os.listdir(model_dir)

['model_schema.json', 'mlp.keras', '.ipynb_checkpoints']

In [7]:
from tensorflow.keras.models import load_model
# Load the model
model = load_model(os.path.join(model_dir,'mlp.keras' ))#+ '/lstm_model.h5'))

In [44]:
initial_sequence = np.array(dataset.close_price)[::-1]

array([46501.89, 46298.02, 46318.01, 46388.21, 46452.17, 46479.63,
       46617.  , 46553.27, 46484.01, 46522.15, 46609.99, 46479.16,
       46481.11, 46438.27, 46374.28, 46339.49, 46534.02, 46553.  ,
       46600.  , 46612.98, 46701.21, 46581.99, 46692.3 , 46747.87])

In [45]:
def predict_hours(model,initial_sequence, h = 6):
    recursive_preds = []
    for i in range(h * 12):
        input_ = initial_sequence.reshape(1,n_steps)
        pred = model.predict(input_,verbose = 0)[0][0]
        recursive_preds.append(pred)
        initial_sequence = np.append(initial_sequence,pred)[-n_steps:]
        #print(initial_sequence)
    return recursive_preds
preds = predict_hours(model, initial_sequence, h = 1)

[46755.996,
 46785.223,
 46811.35,
 46847.332,
 46868.508,
 46903.176,
 46929.86,
 46958.582,
 46986.824,
 47019.38,
 47051.113,
 47075.043]

In [114]:
from datetime import datetime, timedelta
dates = [ int((pd.to_datetime(dataset['open_time'][0],unit = 'ms') + timedelta(minutes = 5 * i)).timestamp() * 1000) for i in range(1,13)]


In [129]:
df = pd.DataFrame(data = {'price_predictions':preds,'timestamp':dates})
df

,price_predictions,timestamp
0,46755.996094,1705003500000
1,46785.222656,1705003800000
2,46811.351562,1705004100000
3,46847.332031,1705004400000
4,46868.507812,1705004700000
5,46903.175781,1705005000000
6,46929.859375,1705005300000
7,46958.582031,1705005600000
8,46986.824219,1705005900000
9,47019.378906,1705006200000


In [130]:
feature_group = FeatureGroup(name = 'btc-price-predictions')
feature_group.ingest(data_frame=df, max_workers=2, wait=True)

2024-01-11 21:38:08,239 INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2024-01-11 21:38:08,454 INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2024-01-11 21:38:08,456 INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2024-01-11 21:38:09,310 INFO: Started ingesting index 6 to 12
2024-01-11 21:38:09,312 INFO: Started ingesting index 0 to 6
2024-01-11 21:38:09,481 INFO: Successfully ingested row 0 to 6
2024-01-11 21:38:09,522 INFO: Successfully ingested row 6 to 12


IngestionManagerPandas(feature_group_name='btc-price-predictions', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f6c4b94a470>, sagemaker_session=<sagemaker.session.Session object at 0x7f6787c889d0>, max_workers=2, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f6b2d1ce7d0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])